In [13]:
import pandas as pd
import requests
import time


In [14]:

# Load your data
df = pd.read_csv('test.csv')

# Load NOAA API token from file
with open('API_token.txt', 'r') as token_file:
    NOAA_TOKEN = token_file.read().strip()

print("NOAA Token loaded successfully: " + NOAA_TOKEN)




NOAA Token loaded successfully: xoMBEjEaGDOQpPljQjueOOokNPPpmFqN


In [15]:

BASE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
HEADERS = {"token": NOAA_TOKEN}

def fetch_weather_data(lat, lon, start_date, end_date, datasetid="GHCND", datatypeid="TAVG", limit=1000):
    # Find nearest station first (you could also cache this)
    stations_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/stations"
    params = {
        "datasetid": datasetid,
        "limit": 1,
        "latitude": lat,
        "longitude": lon,
    }
    response = requests.get(stations_url, headers=HEADERS, params=params)
    if response.status_code != 200:
        print("Station query failed with:", response.status_code)
        print("URL:", response.url)
        print("Response:", response.text)
        response.raise_for_status()
    
    stations = response.json().get('results')
    if not stations:
        print("No stations found near:", lat, lon)
        return None
    
    station_id = response.json()['results'][0]['id']
    
    # Now get the weather data
    data_params = {
        "datasetid": datasetid,
        "datatypeid": datatypeid,
        "stationid": station_id,
        "startdate": start_date,
        "enddate": end_date,
        "limit": limit,
        "units": "metric"
    }
    
    response = requests.get(BASE_URL, headers=HEADERS, params=data_params)
    if response.status_code != 200:
        print("Weather data fetch failed:", response.text)
        return None

    return response.json().get("results", [])


In [17]:
# Sample query to test the function
sample_lat = 43.0731
sample_lon = -89.4012 #madison, WI
sample_start_date = "2023-01-01"
sample_end_date = "2023-01-31"

# Fetch weather data for the sample query
sample_weather_data = fetch_weather_data(
    lat=sample_lat,
    lon=sample_lon,
    start_date=sample_start_date,
    end_date=sample_end_date
)

# Print the sample weather data
print("Sample Weather Data:", sample_weather_data)

Sample Weather Data: []


In [ ]:
# Example: process the DataFrame row by row and add results
weather_data = []

for _, row in df.iterrows():
    result = fetch_weather_data(
        lat=row["latitude"],
        lon=row["longitude"],
        start_date=row["start_date"],
        end_date=row["end_date"]
    )
    weather_data.append(result)
    time.sleep(1)  # Avoid rate limiting

df["weather"] = weather_data

# Save to new file
df.to_csv("weather_data_filled.csv", index=False)